In [1]:
import os
import uuid
import aiohttp
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import select, update
from typing import Type, List

from app.database.models import async_main, drop_all_tables, async_session, generate_erd
from app.database.requests.requests import set_const_value, \
    set_student_group, set_topic, get_const_value, set_organization, \
    set_teacher, set_student, get_all, get_student, delete_student, delete_org
from app.database.models import Base, Student, Organization, LLMRegime, \
    Teacher, PersonalExams, PersonalGraphs, PersonalTopics, PersRegime, \
    Topic, ExamSpecification, ExamsXExams, Exam, ExamsXStudentGroups, \
    StudentGroup, StudentGroupsXSubjects, TeachersXStudentGroups, Task, \
    TasksXStudents, TaskType, Contact, Document, Graph, Subject, Difficulty

### Visualize DB:

In [2]:
# generate_erd('erd_schema.png')

### Add necessary field in database:

Execute all cells below for standard setup of DB

In [3]:
load_dotenv()

TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

async def set_dict_values(model: Type[Base], values):
    for val in values:
        await set_const_value(model, val)

async def apply_func_to_list(func, values, **kwargs):
    for val in values:
        await func(name=val, **kwargs)

def orm_list_to_df(orm_list: list | None) -> pd.DataFrame | None:
    if not orm_list:
        return None
    
    model_class = orm_list[0].__class__
    column_names = [column.name for column in model_class.__table__.columns]

    table = []
    for obj in orm_list:
        row = {col: getattr(obj, col) for col in column_names}
        table.append(row)

    return pd.DataFrame(table)

In [4]:
# await drop_all_tables()
# await async_main()

# Subjects
subjects = ["Математика", "Программирование"]
await set_dict_values(Subject, subjects)

# Task types
task_types = ['Выбор одного из списка', 'Выбор нескольких из списка', 'Да/Нет', 'Текстом в жестком формате', 'Свободный текст']
await set_dict_values(TaskType, task_types)

# Difficulties:
difficulties = ["Простая", "Средняя", "Сложная"]
await set_dict_values(Difficulty, difficulties)

# PersRegime:
pers_regimes = ["Персонализированное обучение", "Выбор темы"]
await set_dict_values(PersRegime, pers_regimes)

# LLMRegime:
llm_regimes = ['Ответ с коротким объяснением', 'Только ответы без объяснений', 'Самостоятельная работа', "Ответ с подробным разобором"]
await set_dict_values(LLMRegime, llm_regimes)

# topics:
math_topics = ["Алгебра", "Геометрия"]
programming_topics = ["Алгоритмы", "Структуры данных", "ООП"]
await apply_func_to_list(set_topic, math_topics, subject_id=1)  # "Математика"
await apply_func_to_list(set_topic, programming_topics, subject_id=2)  # "Программирование"

2025-03-12 16:44:13,430 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-12 16:44:13,430 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 16:44:13,432 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-12 16:44:13,432 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 16:44:13,433 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-12 16:44:13,433 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 16:44:13,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:44:13,434 INFO sqlalchemy.engine.Engine SET session_replication_role = 'replica';
2025-03-12 16:44:13,434 INFO sqlalchemy.engine.Engine [generated in 0.00020s] ()
2025-03-12 16:44:13,435 INFO sqlalchemy.engine.Engine 
            SELECT tablename FROM pg_tables 
            WHERE schemaname = 'public';
        
2025-03-12 16:44:13,436 INFO sqlalchemy.engine.Engine [generated in 0.00015s] ()
2025-03-12 16:44:13,438 INFO sqlalchemy.engine.Engine DROP TABLE IF EXIST

In [5]:
# Test organization
org_invite_code = uuid.uuid4()
await set_organization(org_invite_code, 'Юрахно Константин Феликсович (админ)', "г. Москва, проспект Вернадского, д.41")

# Test teacher
teacher_invite_code = uuid.uuid4()
await set_teacher(teacher_invite_code, 1, 1, "Юрахно Константин Феликсович")

# Test Student groups
student_group_names = ["10A", "10B", "11A", "11B"]
await apply_func_to_list(set_student_group, student_group_names, organization_id=1, teacher_id=1)

# Test student
student_invite_code = uuid.uuid4()
await set_student(student_invite_code, student_group_id=2, name="Юрахно Константин Феликсович", grade=11, organization_id=1)

student_invite_code_2 = uuid.uuid4()
await set_student(student_invite_code_2, student_group_id=2, name="Student_3", grade=11, organization_id=1)

2025-03-12 16:44:22,733 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:44:22,734 INFO sqlalchemy.engine.Engine SELECT organizations.id, organizations.invite_code, organizations.tg_id, organizations.name, organizations.legal_address, organizations.quote, organizations.class_quote 
FROM organizations 
WHERE organizations.invite_code = $1::UUID
2025-03-12 16:44:22,735 INFO sqlalchemy.engine.Engine [generated in 0.00069s] (UUID('25cb06c4-6c6c-4cce-802e-191fbd8b5db1'),)
2025-03-12 16:44:22,738 INFO sqlalchemy.engine.Engine INSERT INTO organizations (invite_code, tg_id, name, legal_address, quote, class_quote) VALUES ($1::UUID, $2::BIGINT, $3::VARCHAR, $4::VARCHAR, $5::INTEGER, $6::INTEGER) RETURNING organizations.id
2025-03-12 16:44:22,738 INFO sqlalchemy.engine.Engine [generated in 0.00030s] (UUID('25cb06c4-6c6c-4cce-802e-191fbd8b5db1'), None, 'Юрахно Константин Феликсович (админ)', 'г. Москва, проспект Вернадского, д.41', None, None)
2025-03-12 16:44:22,739 INFO sqlalchemy.e

In [8]:
objects = await get_all(Student)
objects_df = orm_list_to_df(objects)
display(objects_df)

2025-03-12 16:53:08,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:53:08,168 INFO sqlalchemy.engine.Engine SELECT students.id, students.invite_code, students.tg_id, students.student_group_id, students.organization_id, students.name, students.birth_date, students.grade, students.phone_number, students.pers_regime_id, students.topic_id, students.difficulty_id, students.task_type_id, students.regime_id 
FROM students
2025-03-12 16:53:08,168 INFO sqlalchemy.engine.Engine [cached since 484.1s ago] ()
2025-03-12 16:53:08,171 INFO sqlalchemy.engine.Engine ROLLBACK


,id,invite_code,tg_id,student_group_id,organization_id,name,birth_date,grade,phone_number,pers_regime_id,topic_id,difficulty_id,task_type_id,regime_id
0,2,3d2885d8-c45b-411a-b150-4dd7762d5785,NaN,2,1,Student_3,None,11,None,1,1,1,1,1
1,1,60a99b4a-ef22-493d-bcb3-8120a6361ad1,738490613.0,2,1,Юрахно Константин Феликсович,2002-11-23,11,79017913349,1,4,3,5,4


In [ ]:
await delete_org(7953322626)

In [ ]:
org_invite_code = uuid.uuid4()
await set_organization(org_invite_code, 'ОРГ_"', "АДРЕС_")

In [7]:
classes = [Student, Organization, Teacher, PersRegime, Topic, TaskType, Difficulty, LLMRegime, StudentGroup]
for class_type in classes:
    objects = await get_all(class_type)
    objects_df = orm_list_to_df(objects)
    display(objects_df)

2025-03-12 16:45:04,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:45:04,106 INFO sqlalchemy.engine.Engine SELECT students.id, students.invite_code, students.tg_id, students.student_group_id, students.organization_id, students.name, students.birth_date, students.grade, students.phone_number, students.pers_regime_id, students.topic_id, students.difficulty_id, students.task_type_id, students.regime_id 
FROM students
2025-03-12 16:45:04,107 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ()
2025-03-12 16:45:04,108 INFO sqlalchemy.engine.Engine ROLLBACK


,id,invite_code,tg_id,student_group_id,organization_id,name,birth_date,grade,phone_number,pers_regime_id,topic_id,difficulty_id,task_type_id,regime_id
0,1,60a99b4a-ef22-493d-bcb3-8120a6361ad1,None,2,1,Юрахно Константин Феликсович,None,11,None,1,1,1,1,1
1,2,3d2885d8-c45b-411a-b150-4dd7762d5785,None,2,1,Student_3,None,11,None,1,1,1,1,1


2025-03-12 16:45:04,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:45:04,118 INFO sqlalchemy.engine.Engine SELECT organizations.id, organizations.invite_code, organizations.tg_id, organizations.name, organizations.legal_address, organizations.quote, organizations.class_quote 
FROM organizations
2025-03-12 16:45:04,118 INFO sqlalchemy.engine.Engine [generated in 0.00033s] ()
2025-03-12 16:45:04,120 INFO sqlalchemy.engine.Engine ROLLBACK


,id,invite_code,tg_id,name,legal_address,quote,class_quote
0,1,25cb06c4-6c6c-4cce-802e-191fbd8b5db1,None,Юрахно Константин Феликсович (админ),"г. Москва, проспект Вернадского, д.41",None,None


2025-03-12 16:45:04,122 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:45:04,123 INFO sqlalchemy.engine.Engine SELECT teachers.id, teachers.invite_code, teachers.tg_id, teachers.organization_id, teachers.subject_id, teachers.name, teachers.birth_date, teachers.phone_number, teachers.personal_info 
FROM teachers
2025-03-12 16:45:04,124 INFO sqlalchemy.engine.Engine [generated in 0.00027s] ()
2025-03-12 16:45:04,125 INFO sqlalchemy.engine.Engine ROLLBACK


...

,id,name
0,1,Простая
1,2,Средняя
2,3,Сложная


2025-03-12 16:45:04,156 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:45:04,156 INFO sqlalchemy.engine.Engine SELECT llm_regimes.id, llm_regimes.name 
FROM llm_regimes
2025-03-12 16:45:04,157 INFO sqlalchemy.engine.Engine [generated in 0.00029s] ()
2025-03-12 16:45:04,158 INFO sqlalchemy.engine.Engine ROLLBACK


,id,name
0,1,Ответ с коротким объяснением
1,2,Только ответы без объяснений
2,3,Самостоятельная работа
3,4,Ответ с подробным разобором


2025-03-12 16:45:04,159 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 16:45:04,160 INFO sqlalchemy.engine.Engine SELECT student_groups.id, student_groups.name, student_groups.organization_id, student_groups.teacher_id 
FROM student_groups
2025-03-12 16:45:04,160 INFO sqlalchemy.engine.Engine [generated in 0.00022s] ()
2025-03-12 16:45:04,164 INFO sqlalchemy.engine.Engine ROLLBACK


,id,name,organization_id,teacher_id
0,1,10A,1,1
1,2,10B,1,1
2,3,11A,1,1
3,4,11B,1,1


Read and add to database real teachers from excel file:

Read and add to database real students from excel file:

In [70]:
new_triple = await generate()

In [73]:
task, answer, explanation = new_triple

In [74]:
print(task, answer, explanation, sep='\n')

Верно ли, что уравнение \( x^2 + 5x + 6 = 0 \) имеет два различных действительных корня?
Да
Для проверки верности утверждения необходимо вычислить дискриминант квадратного уравнения \( D = b^2 - 4ac \). В данном случае \( a = 1 \), \( b = 5 \), \( c = 6 \), поэтому \( D = 5^2 - 4*1*6 = 25 - 24 = 1 \). Поскольку дискриминант \( D > 0 \), уравнение имеет два различных действительных корня.


In [94]:
import requests

URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
CATALOGUE_ID =  os.getenv("YC_CATALOGUE_ID")
IAM_TOKEN = os.getenv("IAM_TOKEN")
user_text = "Сгенерируй задачу по математике с решением для ученика 11 класса"

def run(iam_token, catalogue_id, user_text):    
    data = {
        "modelUri": f"gpt://{catalogue_id}/yandexgpt-32k/latest",
        "completionOptions": {"temperature": 0.3, "maxTokens": 1000},
        "messages": [
            {"role": "system", "text": "Ты профессиональный репетитор по математике, который помогает 11-класникам подготовиться к экзамену."},
            {"role": "user", "text": f"{user_text}"},
            ]
        }
    response = requests.post(
        URL,
        headers={
            "Accept": "application/json",
            "Authorization": f"Bearer {iam_token}"
        },
        json=data,
    )
    result = response.json()

    return result

In [95]:
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(
    folder_id=CATALOGUE_ID,
    auth=IAM_TOKEN,
)

model = sdk.models.completions("yandexgpt")
model = model.configure(temperature=0.5)
result = model.run("Что такое небо?")

RuntimeError: no explicit authorization data was passed and no authorization data was found at environment

In [ ]:
result